<a href="https://colab.research.google.com/github/j2cry/hw_DataScienePyLibs/blob/main/hw3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score as r2
from sklearn.metrics import roc_auc_score as auc
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

# **Задание 1**
Импортируйте библиотеки pandas и numpy.
Загрузите "Boston House Prices dataset" из встроенных наборов данных библиотеки sklearn. Создайте датафреймы X и y из этих данных.<br>
Разбейте эти датафреймы на тренировочные (X_train, y_train) и тестовые (X_test, y_test) с помощью функции train_test_split так, чтобы размер тестовой выборки
составлял 30% от всех данных, при этом аргумент random_state должен быть равен 42.<br>
Создайте модель линейной регрессии под названием lr с помощью класса LinearRegression из модуля sklearn.linear_model.<br>
Обучите модель на тренировочных данных (используйте все признаки) и сделайте предсказание на тестовых.<br>
Вычислите R2 полученных предказаний с помощью r2_score из модуля sklearn.metrics.


In [ ]:
boston = load_boston()
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = pd.DataFrame(boston.target, columns=['price'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
r2(y_test, y_pred)

0.7112260057484974

# **Задание 2**
Создайте модель под названием `model` с помощью RandomForestRegressor из модуля sklearn.ensemble.<br>
Сделайте агрумент `n_estimators` равным 1000, `max_depth` должен быть равен 12 и `random_state` сделайте равным 42.<br>
Обучите модель на тренировочных данных аналогично тому, как вы обучали модель LinearRegression, но при этом в метод fit вместо датафрейма y_train поставьте y_train.values[:, 0], чтобы получить из датафрейма одномерный массив Numpy,
так как для класса RandomForestRegressor в данном методе для аргумента `y` предпочтительно применение массивов вместо датафрейма.<br>
Сделайте предсказание на тестовых данных и посчитайте R2. Сравните с результатом из предыдущего задания.<br>
Напишите в комментариях к коду, какая модель в данном случае работает лучше.


In [ ]:
model = RandomForestRegressor(n_estimators=1000, max_depth=12, random_state=42)

model.fit(X_train, y_train.values.flatten())
y_pred = model.predict(X_test)
r2(y_test, y_pred)

0.8749965273218174

очевидно, что `RandomForestRegressor` работает ощутимо лучше

# ***Задание 3**
Вызовите документацию для класса RandomForestRegressor, найдите информацию об атрибуте feature_importances_. С помощью этого атрибута найдите сумму всех показателей важности, установите, какие два признака показывают наибольшую важность.


In [ ]:
RandomForestRegressor?
model.feature_importances_.sum()

0.9999999999999999

In [ ]:
# текстовый вариант
feat_imp = pd.Series(model.feature_importances_, X_train.columns)
feat_imp.sort_values(ascending=False)[:2]

LSTAT    0.415679
RM       0.402705
dtype: float64

In [ ]:
# визуальный вариант
plt.barh(X_train.columns, model.feature_importances_)
plt.xlabel('Weight')
plt.title('Features weights')
plt.show()

# ***Задание 4**
В этом задании мы будем работать с датасетом, с которым мы уже знакомы по домашнему заданию по библиотеке Matplotlib, это датасет Credit Card Fraud Detection.Для этого датасета мы будем решать задачу классификации - будем определять,какие из транзакциции по кредитной карте являются мошенническими.Данный датасет сильно несбалансирован (так как случаи мошенничества относительно редки),так что применение метрики accuracy не принесет пользы и не поможет выбрать лучшую модель.Мы будем вычислять AUC, то есть площадь под кривой ROC.<br>
Импортируйте из соответствующих модулей `RandomForestClassifier`, `GridSearchCV` и `train_test_split`.<br>
Загрузите датасет creditcard.csv и создайте датафрейм df.<br>
С помощью метода value_counts с аргументом `normalize=True` убедитесь в том, что выборка несбалансирована.Используя метод info, проверьте, все ли столбцы содержат числовые данные и нет ли в них пропусков.Примените следующую настройку, чтобы можно было просматривать все столбцы датафрейма:
`pd.options.display.max_columns = 100`.
Просмотрите первые 10 строк датафрейма df.
Создайте датафрейм X из датафрейма df, исключив столбец `Class`.
Создайте объект `Series` под названием `y` из столбца `Class`.
Разбейте X и y на тренировочный и тестовый наборы данных при помощи функции `train_test_split`, используя аргументы: `test_size=0.3`, `random_state=100`, `stratify=y`.
У вас должны получиться объекты X_train, X_test, y_train и y_test.
Просмотрите информацию о их форме.
Для поиска по сетке параметров задайте такие параметры:
`parameters = [{'n_estimators': [10, 15],
'max_features': np.arange(3, 5),
'max_depth': np.arange(4, 7)}]`<br>
Создайте модель GridSearchCV со следующими аргументами:
`estimator=RandomForestClassifier(random_state=100),
param_grid=parameters,
scoring='roc_auc',
cv=3`.<br>
Обучите модель на тренировочном наборе данных (может занять несколько минут).
Просмотрите параметры лучшей модели с помощью атрибута best_params_.
Предскажите вероятности классов с помощью полученнной модели и метода predict_proba.<br>
Из полученного результата (массив Numpy) выберите столбец с индексом 1 (вероятность класса 1) и запишите в массив y_pred_proba. Из модуля sklearn.metrics импортируйте метрику roc_auc_score.<br>
Вычислите AUC на тестовых данных и сравните с результатом,полученным на тренировочных данных, используя в качестве аргументов массивы y_test и y_pred_proba.


In [ ]:
# import data
DATA_PATH = '/content/drive/MyDrive/creditcard.csv'
df = pd.read_csv(DATA_PATH)
pd.options.display.max_columns = 100

In [ ]:
df.value_counts(normalize=True)

In [ ]:
df.info()

In [ ]:
df.head(10)

In [ ]:
X = df.drop(columns='Class')
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100, stratify=y)

In [ ]:
X_train.shape, X_test.shape, df.shape

In [ ]:
parameters = [{'n_estimators': [10, 15], 'max_features': np.arange(3, 5), 'max_depth': np.arange(4, 7)}]

model = GridSearchCV(estimator=RandomForestClassifier(random_state=100), param_grid=parameters, scoring='roc_auc', cv=3)
model.fit(X_train, y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=100,
                                 

In [ ]:
model.best_params_

{'max_depth': 6, 'max_features': 3, 'n_estimators': 15}

In [ ]:
y_train_proba = model.predict_proba(X_train)[:, 1]
y_pred_proba = model.predict_proba(X_test)[:, 1]

In [ ]:
auc(y_test, y_pred_proba), auc(y_train, y_train_proba)

(0.9462664156037156, 0.9703527882554751)